In [ ]:
!pip uninstall -y gym gymnasium ale-py autorom
!pip install "gymnasium[atari]" "ale-py" "autorom[accept-rom-license]" stable-baselines3[extra]
!python -m AutoROM --accept-license

Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Successfully uninstalled gym-0.25.2
Found existing installation: gymnasium 1.2.1
Uninstalling gymnasium-1.2.1:
  Successfully uninstalled gymnasium-1.2.1
Found existing installation: ale-py 0.11.2
Uninstalling ale-py-0.11.2:
  Successfully uninstalled ale-py-0.11.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 951.1/951.1 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 15.8 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=autorom_accept_rom_license-0.6.1-py3-none-any.whl size=446709 sha256=159342c2511c99ebb0ad089428ad8137213628101ad54386b77cda89bd75278d
  Stored in directory

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
All models and videos will be saved to: /content/drive/MyDrive/Pong_RL_Project


In [ ]:
import gymnasium as gym, ale_py
print("Gymnasium version:", gym.__version__)
print("ALE-Py version:", ale_py.__version__)

env = gym.make("ALE/Pong-v5", render_mode="rgb_array")
obs, info = env.reset()
print("Observation shape:", obs.shape)
print("Action space:", env.action_space)
env.close()


Gymnasium version: 1.2.1
ALE-Py version: 0.11.2
Observation shape: (210, 160, 3)
Action space: Discrete(6)


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback

# Create environment
env = DummyVecEnv([lambda: gym.make("ALE/Pong-v5", render_mode="rgb_array")])

# Checkpoint callback - saves every 100k timesteps
checkpoint_callback = CheckpointCallback(
    save_freq=100000,
    save_path="/content/drive/MyDrive/pong_checkpoints/",
    name_prefix="ppo_pong"
)

# Create model
model = PPO("CnnPolicy", env, verbose=1)

# Train model (5 million timesteps recommended, but start small)
model.learn(total_timesteps=500000, callback=checkpoint_callback)

# Save final model
model.save("/content/drive/MyDrive/pong_checkpoints/ppo_pong_final")
env.close()


Using cpu device
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 916      |
|    ep_rew_mean     | -19.5    |
| time/              |          |
|    fps             | 89       |
|    iterations      | 1        |
|    time_elapsed    | 22       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 897         |
|    ep_rew_mean          | -19.8       |
| time/                   |             |
|    fps                  | 15          |
|    iterations           | 2           |
|    time_elapsed         | 265         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010199155 |
|    clip_fraction        | 0.0862      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.78       |
|    explained

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

env = DummyVecEnv([lambda: gym.make("ALE/Pong-v5", render_mode="rgb_array")])

# Replace with your last saved step
model_path = "/content/drive/MyDrive/pong_checkpoints/ppo_pong_400000_steps.zip"

model = PPO.load(model_path)
model.set_env(env)

# Continue training
model.learn(total_timesteps=2000000)
model.save("/content/drive/MyDrive/pong_checkpoints/ppo_pong_continued")
env.close()


NameError: name 'log_dir' is not defined

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from gymnasium.wrappers import RecordVideo
import os

# Create video folder
video_folder = "/content/drive/MyDrive/pong_videos/"
os.makedirs(video_folder, exist_ok=True)

# Load trained model
model = PPO.load("/content/drive/MyDrive/pong_checkpoints/ppo_pong_final")

# Create environment with video recording
env = gym.make("ALE/Pong-v5", render_mode="rgb_array")
env = RecordVideo(env, video_folder=video_folder, episode_trigger=lambda e: True)

obs, info = env.reset()
for _ in range(5000):
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        obs, info = env.reset()

env.close()
print(f"🎥 Video saved in {video_folder}")
